# **Proyek Klasifikasi Gambar CNN: Scientific Images**

## **Tujuan**
Membangun, melatih, dan mengevaluasi model Convolutional Neural Network (CNN) untuk mengklasifikasikan gambar ilmiah ke dalam 6 kategori (Blot-Gel, FACS, Histopathology, Macroscopy, Microscopy, Non-scientific) dengan akurasi minimal 95% pada data training dan testing, serta memenuhi semua kriteria yang ditetapkan.

---

## **Langkah 1: Pengaturan Lingkungan dan Unduh Dataset**
Pertama, kita perlu menyiapkan lingkungan kerja dan mengunduh dataset dari Kaggle.

In [ ]:
# Instal library yang mungkin belum ada (jalankan di terminal atau cell notebook)
# !pip install tensorflow numpy matplotlib split-folders kaggle

import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import splitfolders # Library untuk membagi dataset
import pathlib
import PIL # Python Imaging Library untuk cek gambar

print("TensorFlow Version:", tf.__version__)

In [ ]:
# Konfigurasi Kaggle (jika belum)
# Anda perlu mengunggah file kaggle.json Anda
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Unduh dataset dari Kaggle
# !kaggle datasets download -d rushilprajapati/data-final -p ./data --unzip

Dataset URL: https://www.kaggle.com/datasets/rushilprajapati/data-final
License(s): apache-2.0


In [ ]:
# Tentukan path utama dataset setelah di-unzip
# Sesuaikan path ini jika Anda meletakkannya di lokasi berbeda
dataset_base_dir = '/data' # Ganti jika perlu
dataset_base_path = pathlib.Path(dataset_base_dir)

### **Penjelasan Langkah 1:**
1.  **Instalasi & Impor:** Memasang dan mengimpor library esensial seperti TensorFlow, NumPy, Matplotlib, dan `splitfolders`.
2.  **Kaggle API:** Menyiapkan Kaggle API untuk mengunduh dataset (komentar `#` bisa dihilangkan jika dijalankan di lingkungan seperti Colab).
3.  **Unduh & Unzip:** Mengunduh dataset dari URL Kaggle yang diberikan dan mengekstraknya.

---

## **Langkah 2: Eksplorasi Data Analisis (EDA)**

In [ ]:
# --- Eksplorasi Awal: Cek Kelas dan Tampilkan Sampel ---
print("\n--- Eksplorasi Data Awal ---")
class_names = sorted([item.name for item in dataset_base_path.glob('*') if item.is_dir()])
num_classes = len(class_names)
print(f"Kelas yang ditemukan ({num_classes}): {class_names}")

In [ ]:
# Tampilkan 2 sampel per kelas
plt.figure(figsize=(12, 8))
print("Menampilkan 2 sampel gambar per kelas:")
for i, class_name in enumerate(class_names):
    class_dir = dataset_base_path / class_name
    # Ambil 2 file gambar (jpg atau png)
    sample_files = list(class_dir.glob('*.jpg'))[:2] + list(class_dir.glob('*.png'))[:2]
    sample_files = sample_files[:2] # Pastikan hanya 2

    if not sample_files:
        print(f"  Peringatan: Tidak ada sampel ditemukan untuk kelas '{class_name}'")
        continue

    for j, file_path in enumerate(sample_files):
        plt.subplot(num_classes, 2, i * 2 + j + 1)
        try:
            img = PIL.Image.open(file_path)
            plt.imshow(img)
            plt.title(f"Kelas: {class_name}", fontsize=10)
            plt.axis('off')
        except Exception as e:
            print(f"Error membuka {file_path}: {e}")
            plt.title(f"Error: {class_name}", fontsize=10)
            plt.axis('off')

plt.tight_layout()
plt.show()

## **Langkah 3: Pembagian Dataset (Train, Validation, Test Set)**
Kita akan membagi dataset menjadi tiga bagian: training (misal 70%), validation (misal 15%), dan testing (misal 15%) menggunakan library `splitfolders`.

In [ ]:
# Tentukan direktori output untuk dataset yang sudah dibagi
output_dir = 'data/scientific_images_split'

In [ ]:
# Hapus direktori output jika sudah ada untuk memastikan pembagian bersih
if os.path.exists(output_dir):
    import shutil
    shutil.rmtree(output_dir)
    print(f"Direktori '{output_dir}' yang sudah ada dihapus.")

In [ ]:
# Membagi dataset (70% train, 15% val, 15% test)
# splitfolders akan membuat struktur folder train/val/test secara otomatis
print(f"Membagi dataset dari '{dataset_base_dir}' ke '{output_dir}'...")
splitfolders.ratio(
    dataset_base_dir,
    output=output_dir,
    seed=42, # Untuk reproduktifitas
    ratio=(.7, .15, .15), # Rasio train/val/test
    group_prefix=None, # Tidak ada prefix grup file
    move=False # Salin file, jangan pindahkan (lebih aman)
)

print("Pembagian dataset selesai.")
print(f"Dataset terbagi tersedia di: {output_dir}")

In [ ]:
# Definisikan path untuk setiap set
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

In [ ]:
# Verifikasi jumlah gambar di setiap set
def count_files(directory):
    count = 0
    for root, _, files in os.walk(directory):
        count += len([f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return count

print(f"Jumlah gambar training: {count_files(train_dir)}")
print(f"Jumlah gambar validation: {count_files(val_dir)}")
print(f"Jumlah gambar testing: {count_files(test_dir)}")

### **Penjelasan Langkah 3:**
1.  **Output Directory:** Menentukan lokasi penyimpanan dataset yang sudah dibagi.
2.  **Splitfolders:** Menggunakan library `splitfolders` yang praktis untuk membagi dataset berdasarkan rasio yang ditentukan sambil mempertahankan struktur kelas di dalam folder `train`, `val`, dan `test`.
3.  **Verifikasi:** Menghitung jumlah file di setiap set untuk memastikan pembagian berhasil.

---

## **Langkah 4: Preprocessing dan Augmentasi Data**

Kita akan menggunakan `ImageDataGenerator` (atau `tf.keras.utils.image_dataset_from_directory` sebagai alternatif modern) untuk:
1.  Membaca gambar dari direktori.
2.  Mengubah ukuran semua gambar ke ukuran yang seragam untuk menangani ukuran gambar yang beragam.
3.  Normalisasi nilai piksel (biasanya ke rentang [0, 1]).
4.  Menerapkan augmentasi data pada set pelatihan untuk meningkatkan robustisitas model dan mengurangi overfitting.

In [ ]:
# Membuat dataset train, validation, test menggunakan tf.data
print("\nMembuat pipeline data tf.data...")

# Tentukan parameter
IMG_HEIGHT = 224 # Tinggi gambar
IMG_WIDTH = 224 # Lebar gambar
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 128 # Sesuaikan berdasarkan memori GPU

In [ ]:
# Load data dari direktori ke tf.data.Dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical' # Penting untuk CategoricalCrossentropy
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    shuffle=False, # Tidak perlu shuffle validation/test
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

In [ ]:
# Pastikan nama kelas konsisten
class_names_tfdata = train_dataset.class_names
print(f"Kelas yang dideteksi oleh tf.data: {class_names_tfdata}")
if class_names != class_names_tfdata:
    print("Peringatan: Urutan kelas berbeda antara eksplorasi awal dan tf.data!")
    # Sebaiknya gunakan class_names_tfdata sebagai referensi utama
    class_names = class_names_tfdata
    num_classes = len(class_names)

In [ ]:
# --- Augmentasi Data sebagai Layer Keras ---
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ],
    name="data_augmentation",
)

In [ ]:
# --- Optimasi Pipeline Data ---
# Normalisasi (dipisah agar bisa diterapkan setelah augmentasi jika augmentasi di luar model)
# atau bisa langsung di model pre-trained jika menggunakan preprocess_input
# Di sini kita normalisasi manual [0, 1]
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

# Terapkan augmentasi dan normalisasi pada training set
# Terapkan normalisasi saja pada validation dan test set
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.map(normalize_img, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)

validation_dataset = validation_dataset.map(normalize_img, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

test_dataset = test_dataset.map(normalize_img, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print("Pipeline tf.data dengan augmentasi (train) dan prefetching siap.")

### **Penjelasan Langkah 4:**
1.  **Parameter:** Menentukan ukuran gambar input (`IMG_WIDTH`, `IMG_HEIGHT`) dan ukuran batch (`BATCH_SIZE`). Ukuran gambar yang seragam untuk menangani masalah ukuran gambar yang berbeda-beda.
2.  **ImageDataGenerator:** Membuat dua instance: satu untuk training dengan augmentasi (rotasi, geser, zoom, flip) dan satu lagi untuk validasi/testing hanya dengan normalisasi.
3.  **Data Generators:** Menggunakan metode `flow_from_directory` untuk membuat generator yang akan membaca gambar dari folder, melakukan preprocessing/augmentasi, dan menyajikannya dalam batch ke model. `class_mode='categorical'` digunakan karena ini adalah masalah klasifikasi multi-kelas.
---

## **Langkah 5: Membangun Model CNN (Sequential, Conv2D, Pooling)**

Kita akan membangun model CNN menggunakan Keras Sequential API, yang terdiri dari lapisan `Conv2D` dan `MaxPooling2D`.

In [ ]:
# Membangun Model Sequential
model = models.Sequential([
    # Lapisan Konvolusi Pertama
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),

    # Lapisan Konvolusi Kedua
    layers.Conv2D(64, (3, 3), activation='relu', ),
    layers.MaxPooling2D((2, 2)),

    # Lapisan Konvolusi Ketiga
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Lapisan Konvolusi Keempat (Opsional, bisa ditambah/dikurangi sesuai kebutuhan)
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Meratakan output untuk lapisan Dense
    layers.Flatten(),

    # Lapisan Dense (Fully Connected)
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5), # Dropout untuk regularisasi

    # Lapisan Output
    layers.Dense(num_classes, activation='softmax') # Softmax untuk multi-kelas
])

# Menampilkan ringkasan model
model.summary()

### **Penjelasan Langkah 5:**
1.  **Sequential API:** Model dibangun lapis demi lapis.
2.  **Conv2D:** Lapisan konvolusi mengekstraksi fitur dari gambar menggunakan filter (kernel). Fungsi aktivasi 'relu' umum digunakan. `input_shape` ditentukan hanya di lapisan pertama.
3.  **MaxPooling2D:** Lapisan pooling mengurangi dimensi spasial (tinggi dan lebar) representasi fitur, membuatnya lebih robust terhadap variasi posisi fitur dan mengurangi komputasi.
4.  **Flatten:** Mengubah output dari lapisan konvolusi/pooling (yang berupa tensor 3D) menjadi vektor 1D agar bisa dimasukkan ke lapisan Dense.
5.  **Dense:** Lapisan fully connected untuk melakukan klasifikasi berdasarkan fitur yang diekstraksi.
6.  **Dropout:** Teknik regularisasi untuk mencegah overfitting dengan menonaktifkan sebagian neuron secara acak selama training.
7.  **Output Layer:** Lapisan Dense terakhir dengan jumlah neuron sama dengan jumlah kelas dan fungsi aktivasi `softmax` untuk menghasilkan probabilitas untuk setiap kelas.

### **Langkah 6: Kompilasi Model**

Mengkonfigurasi model untuk training dengan menentukan optimizer, loss function, dan metrik evaluasi.

In [ ]:
# Kompilasi model
model.compile(
    optimizer='adam', # Optimizer yang umum digunakan
    loss='categorical_crossentropy', # Loss function untuk multi-kelas kategorikal
    metrics=['accuracy'] # Metrik untuk evaluasi
)

print("Model berhasil dikompilasi.")

**Penjelasan Langkah 6:**
1.  **Optimizer:** Algoritma yang digunakan untuk memperbarui bobot model berdasarkan loss (misalnya, 'adam', 'rmsprop', 'sgd').
2.  **Loss Function:** Mengukur seberapa baik performa model pada data training. `categorical_crossentropy` cocok untuk klasifikasi multi-kelas dengan label one-hot encoded (yang secara otomatis dihasilkan oleh `ImageDataGenerator` dengan `class_mode='categorical'`).
3.  **Metrics:** Metrik yang digunakan untuk mengevaluasi performa model selama training dan testing. 'accuracy' adalah pilihan umum untuk klasifikasi.

---